In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [2]:
#funcion delete
def delete_column(df, column_name):
    del df[column_name]
    return "column deleted"

## Import train DF

In [3]:
def conection(database):
    connection = create_engine("sqlite:///"+ database).connect()
    return connection
con = conection("../files/diamonds_train.db")

In [4]:
def query_diamond(query):
    df_diamonds_full_train = pd.read_sql_query(query,con)
    return df_diamonds_full_train

query_diamonds_full_train = '''
SELECT 
    d_prop.index_id,
    
    d_tran.carat,
    d_cut.cut,
    d_col.color,
    d_clar.clarity,

    d_dim.depth,
    d_dim.'table',
    d_dim.x,
    d_dim.y,
    d_dim.z,
    
    d_city.city,

    d_tran.price
    
FROM diamonds_properties as d_prop
    INNER JOIN diamonds_clarity as d_clar ON d_prop.clarity_id = d_clar.clarity_id
    INNER JOIN diamonds_color as d_col ON d_prop.color_id = d_col.color_id
    INNER JOIN diamonds_cut as d_cut ON d_prop.cut_id = d_cut.cut_id
    INNER JOIN diamonds_dimensions as d_dim ON d_prop.index_id = d_dim.index_id
    INNER JOIN diamonds_transactional as d_tran ON d_prop.index_id = d_tran.index_id
    INNER JOIN diamonds_city as d_city ON d_tran.city_id = d_city.city_id
'''

df_diamonds_train = query_diamond(query_diamonds_full_train)

In [5]:
df_diamonds_train.head()

,index_id,carat,cut,color,clarity,depth,table,x,y,z,city,price
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,1.21,Premium,J,VS2,62.4,58.0,6.83,6.79,4.25,Dubai,4268
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,0.32,Very Good,H,VS2,63.0,57.0,4.35,4.38,2.75,Kimberly,505
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,0.71,Fair,G,VS1,65.5,55.0,5.62,5.53,3.65,Las Vegas,2686
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,0.41,Good,D,SI1,63.8,56.0,4.68,4.72,3.00,Kimberly,738
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,1.02,Ideal,G,SI1,60.5,59.0,6.55,6.51,3.95,Dubai,4882


In [6]:
df_diamonds_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index_id  40455 non-null  object 
 1   carat     40455 non-null  float64
 2   cut       40455 non-null  object 
 3   color     40455 non-null  object 
 4   clarity   40455 non-null  object 
 5   depth     40455 non-null  float64
 6   table     40455 non-null  float64
 7   x         40455 non-null  float64
 8   y         40455 non-null  float64
 9   z         40455 non-null  float64
 10  city      40455 non-null  object 
 11  price     40455 non-null  int64  
dtypes: float64(6), int64(1), object(5)
memory usage: 3.7+ MB


In [7]:
def ejecuciondatasetaugmentation(df_diamonds_train):
    df_diamonds_train2 = df_diamonds_train.copy()
    df_diamonds_train2["carat"] = df_diamonds_train2["carat"]+0.01
    df_diamonds_train2["depth"] = df_diamonds_train2["depth"]+0.1
    df_diamonds_train2["table"] = df_diamonds_train2["table"]+0.1
    df_diamonds_train2["x"] = df_diamonds_train2["x"]+0.01
    df_diamonds_train2["y"] = df_diamonds_train2["y"]+0.01
    df_diamonds_train2["z"] = df_diamonds_train2["z"]+0.01
    df_diamonds_train2["price"] = df_diamonds_train2["price"]+1

    df_diamonds_train3 = pd.concat([df_diamonds_train,df_diamonds_train2], axis=0).reset_index(drop=True)

    df_diamonds_train = df_diamonds_train3.copy()
    return df_diamonds_train

In [8]:
df_diamonds_train = ejecuciondatasetaugmentation(df_diamonds_train)
df_diamonds_train = ejecuciondatasetaugmentation(df_diamonds_train)
df_diamonds_train = ejecuciondatasetaugmentation(df_diamonds_train)
df_diamonds_train = ejecuciondatasetaugmentation(df_diamonds_train)
df_diamonds_train = ejecuciondatasetaugmentation(df_diamonds_train)

In [9]:
#df_diamonds_train[df_diamonds_train["x"]<1].index.tolist()

In [10]:
#df_diamonds_train[df_diamonds_train["z"]<1].index.tolist()

In [11]:
#df_diamonds_train.drop(df_diamonds_train[df_diamonds_train["x"]<1].index,axis=0,inplace=True)
#df_diamonds_train.drop(df_diamonds_train[df_diamonds_train["z"]<1].index,axis=0,inplace=True)

In [12]:
#df_diamonds_train[df_diamonds_train["y"]<1].index.tolist()

In [13]:
#df_diamonds_train["z"].min()

In [14]:
#import plotly.express as px
#fig_inic = px.box(df_diamonds_train["price"])
#fig_inic.show()

In [15]:
df_diamonds_train["xy"] = df_diamonds_train["x"]*df_diamonds_train["y"]

In [16]:
df_diamonds_train["dt"] = df_diamonds_train["depth"]/df_diamonds_train["table"]

In [17]:
#df_diamonds_train["caratxy"] = df_diamonds_train["carat"]*df_diamonds_train["x"]*df_diamonds_train["y"]

### quitar outliers

In [18]:
#df_diamonds_train[df_diamonds_train["carat"]>=4]

In [19]:
#df_diamonds_train.drop(df_diamonds_train[df_diamonds_train["carat"]>=4].index,axis=0,inplace=True)

In [20]:
#df_diamonds_train= df_diamonds_train.reset_index(drop=True)
#df_diamonds_train.head()

## Import test DF

In [21]:
def import_csv_diamonds(location):
    df_diamonds_test = pd.read_csv(location)
    return df_diamonds_test

df_diamonds_test = import_csv_diamonds("../files/diamonds_test.csv")

In [22]:
df_diamonds_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam


In [23]:
df_diamonds_test["xy"] = df_diamonds_test["x"]*df_diamonds_test["y"]

In [24]:
df_diamonds_test["dt"] = df_diamonds_test["depth"]/df_diamonds_test["table"]

In [25]:
#df_diamonds_test["caratxy"] = df_diamonds_test["carat"]*df_diamonds_test["x"]*df_diamonds_train["y"]

## X_train, y_train

In [26]:
X_train_cat = df_diamonds_train.copy()
delete_column(X_train_cat, "price")
delete_column(X_train_cat, "index_id")
delete_column(X_train_cat, "z")
y_train_cat = df_diamonds_train["price"]

In [27]:
X_train_cat[["cut","color","clarity","city"]] = X_train_cat[["cut","color","clarity","city"]].astype("category")

X_train_cat["cut"] = X_train_cat["cut"].cat.codes
X_train_cat["color"] = X_train_cat["color"].cat.codes
X_train_cat["clarity"] = X_train_cat["clarity"].cat.codes
X_train_cat["city"] = X_train_cat["city"].cat.codes

In [28]:
X_train_cat["carat"] = X_train_cat["carat"]*10000
X_train_cat["x"] = X_train_cat["x"]*10
X_train_cat["y"] = X_train_cat["y"]*10

In [29]:
X_train_cat["xy"] = X_train_cat["xy"]*10

In [30]:
X_train_cat["carat2"] = X_train_cat["carat"]*100

In [31]:
#X_train_cat["cut"] = X_train_cat["cut"]*10
#X_train_cat["color"] = X_train_cat["color"]*10
#X_train_cat["clarity"] = X_train_cat["clarity"]*10
#X_train_cat["city"] = X_train_cat["city"]*10

In [32]:
X_train_cat.head()

,carat,cut,color,clarity,depth,table,x,y,city,xy,dt,carat2
0,12100.0,3,6,5,62.4,58.0,68.3,67.9,2,463.757,1.075862,1210000.0
1,3200.0,4,4,5,63.0,57.0,43.5,43.8,3,190.530,1.105263,320000.0
2,7100.0,0,3,4,65.5,55.0,56.2,55.3,4,310.786,1.190909,710000.0
3,4100.0,1,0,2,63.8,56.0,46.8,47.2,3,220.896,1.139286,410000.0
4,10200.0,2,3,2,60.5,59.0,65.5,65.1,2,426.405,1.025424,1020000.0


####  scaling - X_train

from sklearn.preprocessing import StandardScaler
X_train_columns = X_train.columns.tolist()
train_scaled = StandardScaler()
train_scaled_fit = train_scaled.fit_transform(X_train[X_train_columns])
X_train_scaled = pd.DataFrame(train_scaled_fit, columns = X_train_columns)
X_train_scaled.head()

## X_test

In [33]:
X_test = df_diamonds_test.copy()
delete_column(X_test, "id")
delete_column(X_test, "z")
X_test

,carat,cut,color,clarity,depth,table,x,y,city,xy,dt
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,Amsterdam,34.2798,1.045000
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,Surat,46.9209,1.070175
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,Kimberly,54.0216,1.019672
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,Kimberly,37.3317,1.181481
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,Amsterdam,25.7045,1.084483
...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,Amsterdam,28.4620,1.105357
13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,New York City,32.7183,1.130909
13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,Tel Aviv,32.8325,1.120000
13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,Surat,34.4565,1.031579


In [34]:
X_test[["cut","color","clarity","city"]] = X_test[["cut","color","clarity","city"]].astype("category")

X_test["cut"] = X_test["cut"].cat.codes
X_test["color"] = X_test["color"].cat.codes
X_test["clarity"] = X_test["clarity"].cat.codes
X_test["city"] = X_test["city"].cat.codes

In [35]:
X_test["carat"] = X_test["carat"]*10000
X_test["x"] = X_test["x"]*10
X_test["y"] = X_test["y"]*10

In [36]:
#X_test["cut"] = X_test["cut"]*10
#X_test["color"] = X_test["color"]*10
#X_test["clarity"] = X_test["clarity"]*10
#X_test["city"] = X_test["city"]*10

In [37]:
X_test["xy"] = X_test["xy"]*10

In [38]:
X_test["carat2"] = X_test["carat"]*100

In [39]:
X_test.head()

,carat,cut,color,clarity,depth,table,x,y,city,xy,dt,carat2
0,7900.0,4,2,2,62.7,60.0,58.2,58.9,0,342.798,1.045000,790000.0
1,12000.0,2,6,4,61.0,57.0,68.1,68.9,10,469.209,1.070175,1200000.0
2,15700.0,3,4,2,62.2,61.0,73.8,73.2,3,540.216,1.019672,1570000.0
3,9000.0,4,2,2,63.8,54.0,60.9,61.3,3,373.317,1.181481,900000.0
4,5000.0,4,2,4,62.9,58.0,50.5,50.9,0,257.045,1.084483,500000.0


####  scaling - X_test

X_test_columns = X_test.columns.tolist()
train_scaled = StandardScaler()
train_scaled_fit = train_scaled.fit_transform(X_test[X_test_columns])
X_test_scaled = pd.DataFrame(train_scaled_fit, columns = X_test_columns)
X_test_scaled.head()

## MODEL

In [40]:
#sin optimizar 
"""from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train_cat, y_train_cat)
predictions = model.predict(X_test).clip(350,18000)
id_predictions = [i for i in range(0,len(predictions))]
predictions_df = pd.DataFrame({"id":id_predictions , "price":predictions })
predictions_df.head()
predictions_df.to_csv("all_variables_cat_codes_manual_escaling_in_numeric_maspesocarat_random_forest.csv", sep=",", index=False)"""

'from sklearn.ensemble import RandomForestRegressor\nmodel = RandomForestRegressor()\nmodel.fit(X_train_cat, y_train_cat)\npredictions = model.predict(X_test).clip(350,18000)\nid_predictions = [i for i in range(0,len(predictions))]\npredictions_df = pd.DataFrame({"id":id_predictions , "price":predictions })\npredictions_df.head()\npredictions_df.to_csv("all_variables_cat_codes_manual_escaling_in_numeric_maspesocarat_random_forest.csv", sep=",", index=False)'

In [41]:
#sin optimizar 
"""from xgboost.sklearn import XGBClassifier
model = XGBClassifier()
model.fit(X_train_cat, y_train_cat)
predictions = model.predict(X_test).clip(350,18000)
id_predictions = [i for i in range(0,len(predictions))]
predictions_df = pd.DataFrame({"id":id_predictions , "price":predictions })
predictions_df.head()
predictions_df.to_csv("all_variables_cat_codes_manual_escaling_in_numeric_maspesocarat_xgbclassifier.csv", sep=",", index=False)"""

'from xgboost.sklearn import XGBClassifier\nmodel = XGBClassifier()\nmodel.fit(X_train_cat, y_train_cat)\npredictions = model.predict(X_test).clip(350,18000)\nid_predictions = [i for i in range(0,len(predictions))]\npredictions_df = pd.DataFrame({"id":id_predictions , "price":predictions })\npredictions_df.head()\npredictions_df.to_csv("all_variables_cat_codes_manual_escaling_in_numeric_maspesocarat_xgbclassifier.csv", sep=",", index=False)'

## XGB OPT - peor modelo en cuanto a prdicción, aunque muy parecido a random forest

In [42]:
"""import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold


X_train,X_test,y_train,y_test = train_test_split(X_train_cat, y_train_cat, test_size=0.2)
xgb_model = xgb.XGBRegressor()

# Define the parameter options for the Grid Search
parameters = {}

# Define the GridSearch class, including cross-validation, using your XGBoost model
clf = GridSearchCV(xgb_model, parameters)#, cv=StratifiedKFold(cv=5, shuffle=True).split(X_train, y_train))

# Fit the model
clf.fit(X_train, y_train)

best_estimator = clf.best_estimator_
best_estimator"""

'import xgboost as xgb\nfrom sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold\n\n\nX_train,X_test,y_train,y_test = train_test_split(X_train_cat, y_train_cat, test_size=0.2)\nxgb_model = xgb.XGBRegressor()\n\n# Define the parameter options for the Grid Search\nparameters = {}\n\n# Define the GridSearch class, including cross-validation, using your XGBoost model\nclf = GridSearchCV(xgb_model, parameters)#, cv=StratifiedKFold(cv=5, shuffle=True).split(X_train, y_train))\n\n# Fit the model\nclf.fit(X_train, y_train)\n\nbest_estimator = clf.best_estimator_\nbest_estimator'

In [43]:
"""from xgboost import XGBRegressor
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, #missing=NaN,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
model.fit(X_train_cat, y_train_cat)#, eval_metric='auc')
predictions = model.predict(X_test).clip(350,18000)
id_predictions = [i for i in range(0,len(predictions))]
predictions_df = pd.DataFrame({"id":id_predictions , "price":predictions })
predictions_df.head()
predictions_df.to_csv("all_variables_cat_codes_manual_escaling_in_numeric_maspesocarat_xgbregressor7_params.csv", sep=",", index=False)"""

'from xgboost import XGBRegressor\nmodel = XGBRegressor(base_score=0.5, booster=\'gbtree\', colsample_bylevel=1,\n             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,\n             gamma=0, gpu_id=-1, importance_type=None,\n             interaction_constraints=\'\', learning_rate=0.300000012,\n             max_delta_step=0, max_depth=6, min_child_weight=1, #missing=NaN,\n             monotone_constraints=\'()\', n_estimators=100, n_jobs=8,\n             num_parallel_tree=1, predictor=\'auto\', random_state=0, reg_alpha=0,\n             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=\'exact\',\n             validate_parameters=1, verbosity=None)\nmodel.fit(X_train_cat, y_train_cat)#, eval_metric=\'auc\')\npredictions = model.predict(X_test).clip(350,18000)\nid_predictions = [i for i in range(0,len(predictions))]\npredictions_df = pd.DataFrame({"id":id_predictions , "price":predictions })\npredictions_df.head()\npredictions_df.to_csv("all_variables_c

## RANDOM FOREST OPT

In [44]:
"""from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}"""

"from sklearn.ensemble import RandomForestRegressor\n\nfrom sklearn.model_selection import RandomizedSearchCV\n# Number of trees in random forest\nn_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]\n# Number of features to consider at every split\nmax_features = ['auto', 'sqrt']\n# Maximum number of levels in tree\nmax_depth = [int(x) for x in np.linspace(10, 110, num = 11)]\nmax_depth.append(None)\n# Minimum number of samples required to split a node\nmin_samples_split = [2, 5, 10]\n# Minimum number of samples required at each leaf node\nmin_samples_leaf = [1, 2, 4]\n# Method of selecting samples for training each tree\nbootstrap = [True, False]\n# Create the random grid\nrandom_grid = {'n_estimators': n_estimators,\n               'max_features': max_features,\n               'max_depth': max_depth,\n               'min_samples_split': min_samples_split,\n               'min_samples_leaf': min_samples_leaf,\n               'bootstrap': bootstrap}"

In [45]:
"""X_train,X_test,y_train,y_test = train_test_split(X_train_cat, y_train_cat, test_size=0.2)
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_"""

'X_train,X_test,y_train,y_test = train_test_split(X_train_cat, y_train_cat, test_size=0.2)\n# Use the random grid to search for best hyperparameters\n# First create the base model to tune\nrf = RandomForestRegressor()\n# Random search of parameters, using 3 fold cross validation, \n# search across 100 different combinations, and use all available cores\nrf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)\n# Fit the random search model\nrf_random.fit(X_train, y_train)\nrf_random.best_params_'

In [46]:
"""from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators= 1400,
 min_samples_split= 2,
 min_samples_leaf= 1,
 max_features= 'auto',
 max_depth= 100,
 bootstrap= True)
model.fit(X_train_cat, y_train_cat)
predictions = model.predict(X_test).clip(350,18000)
id_predictions = [i for i in range(0,len(predictions))]
predictions_df = pd.DataFrame({"id":id_predictions , "price":predictions })
predictions_df.head()
predictions_df.to_csv("all_variables_cat_codes_manual_escaling_in_numeric_maspesocarat_differetnmodels2_no_z_new_columns3_opt_class_hiperparameters.csv", sep=",", index=False)"""

'from sklearn.ensemble import RandomForestRegressor\nmodel = RandomForestRegressor(n_estimators= 1400,\n min_samples_split= 2,\n min_samples_leaf= 1,\n max_features= \'auto\',\n max_depth= 100,\n bootstrap= True)\nmodel.fit(X_train_cat, y_train_cat)\npredictions = model.predict(X_test).clip(350,18000)\nid_predictions = [i for i in range(0,len(predictions))]\npredictions_df = pd.DataFrame({"id":id_predictions , "price":predictions })\npredictions_df.head()\npredictions_df.to_csv("all_variables_cat_codes_manual_escaling_in_numeric_maspesocarat_differetnmodels2_no_z_new_columns3_opt_class_hiperparameters.csv", sep=",", index=False)'

## GridSearchCV()

In [47]:
#from sklearn.ensemble import RandomForestRegressor

#model = RandomForestRegressor(random_state = 42)
#model.fit(X_train_cat, y_train_cat)

In [48]:
#param_grid = {'n_estimators': [16, 32, 64, 128, 256, 512],
#              'max_depth': [2, 4, 8, 16]}

In [49]:
#grid_search = RandomizedSearchCV(model, 
#                                 param_grid, 
#                                 cv=5, 
#                                 verbose=3, 
#                                 scoring='neg_root_mean_squared_error', 
#                                 n_jobs=-1)

In [50]:
#grid_search.fit(X_train_cat,y_train_cat)

#print('\n', 'Best hyperparameters: ', grid_search.best_params_, '\n')
#print('Best score: ', -grid_search.best_score_, '\n')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators= 1024,
 max_depth= 16)
model.fit(X_train_cat, y_train_cat)
predictions = model.predict(X_test).clip(350,18000)
id_predictions = [i for i in range(0,len(predictions))]
predictions_df = pd.DataFrame({"id":id_predictions , "price":predictions })
predictions_df.head()
predictions_df.to_csv("all_variables_cat_codes_manual_escaling_in_numeric_maspesocarat_differetnmodels2_no_z_new_columns3_opt_class_hiperparameters_condatasetamp.csv", sep=",", index=False)